# Claude 3.7 Sonnet と拡張思考の紹介

このノートブックでは、Anthropic の Claude 3.7 Sonnet モデルとその革新的な「拡張思考」機能について紹介します。次の内容について説明します。

1. Claude 3.7 Sonnet の機能の概要
2. 拡張思考とその仕組みの理解
3. Amazon Bedrock での Claude 3.7 のセットアップ
4. 標準モードと拡張思考モードの比較
5. 思考プロセスの視覚化

このノートブックを読み終えると、拡張思考を使用して AI ワークフローを改善するタイミングと方法を明確に理解できるようになります。

In [1]:
!pip install -r requirements.txt -qU --disable-pip-version-check

In [ ]:
# Import required libraries
import boto3
from botocore.config import Config
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from termcolor import colored
from IPython.display import display, Markdown, HTML

## 1. Claude 3.7 Sonnet の機能の概要

Claude 3.7 Sonnet は、2025 年 2 月にリリースされた Anthropic のこれまでで最も高度なモデルです。以前の Claude モデルに比べていくつかの重要な改善が導入されています:

### 主な機能

- **ハイブリッド推論アプローチ**: Claude 3.7 Sonnet は標準モードと「拡張思考」モードの両方で動作できるため、モデルがより深い推論を行うタイミングを制御できます。

- **出力長の増加**: 最大 64K の出力トークン (以前のモデルの 8 倍の長さ) をサポートし、プレビューでは最大 128K のトークンがサポートされています。

- **コンピューターの使用強化**: スクロール、待機、マウスの左下/上、キーのホールド、トリプルクリックなどの追加アクションを含む、コンピューター操作の機能が向上しました。

- **コード生成の改善**: コーディング ベンチマーク、特に SWE-bench Verified でトップクラスのパフォーマンスを発揮します。

- **推論予算の制御**: API を使用すると、タスクに割り当てる「思考力」の量を正確に制御できます。必要な最小限の予算から、複雑な問題に対するはるかに大きな割り当てまで、さまざまな方法で制御できます。

Claude 3.7 Sonnet は、詳細な分析、複雑な問題解決、および複数ステップの推論を必要とするタスクに優れています。また、長時間の思考が必要ない場合は、迅速な応答を提供する機能も維持しています。

## 2. 拡張思考を理解する

### 拡張思考とは何ですか?

拡張思考は、ユーザーに表示できる段階的な推論を通じて、クロードが複雑な問題を解決できる画期的な機能です。最終的な答えを見るのではなく、数学者がホワイトボード上で証明を進めていくのを見ているようなものだと考えてください。

### 拡張思考の仕組み

Claude 3.7 Sonnet を有効にすると、人間の問題解決に似たプロセスが実行されます:

1. 最初にタスクを内部の「スクラッチパッド」で処理します - 問題を段階的に考えます
2. この推論プロセスは、API 応答で確認できます
3. 推論を完了すると、Claude はこの思考に基づいて最終的な回答を提供します

### 拡張思考と思考の連鎖

**従来の思考の連鎖 (CoT):**
- 段階的な推論を引き出すには、特定のプロンプトが必要です
- 推論の品質はプロンプト エンジニアリングに大きく依存します
- 推論と応答が混在しています
- 推論の深さを制御できません

**拡張思考:**
- API パラメーターによって明示的に有効になります
- 推論の予算を正確に制御できます
- 推論は、最終応答とは別のフィールドに表示されます
- より徹底的で構造化された推論

### 推論の予算

Claude の重要な革新3.7 Sonnet は、「推論予算」を制御する機能です。これは、思考プロセスに割り当てられるトークンの量です。

- 最小予算: 1,024 トークン
- モデルの 128K トークン制限まで増やすことができます
- 予算が大きいほど、複雑な問題に対するより徹底した推論が可能になります

推論予算は、計算タスクに CPU 時間を割り当てるようなものと考えてください。より複雑なタスクほど、割り当てが大きいほどメリットがあります。

In [ ]:
## 3. Setting up Claude 3.7 in Amazon Bedrock

# Configure the AWS region
REGION = 'us-west-2'  # Change to your preferred region
config = Config(read_timeout=300)


# Initialize Bedrock clients
bedrock = boto3.client(
    service_name='bedrock',
    region_name=REGION,
    config=config
)

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=REGION,
)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# For comparison, Claude 3.5 Sonnet model ID
CLAUDE_35_SONNET_MODEL_ID = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'

> **注**: このレッスンでは、わかりやすくするために以下の関数を示していますが、今後はこれらのユーティリティ関数を `claude_utils` としてインポートします。`claude_utils.py` モジュールには、Bedrock クライアントを作成し、拡張思考の有無にかかわらず Claude を呼び出し、応答を表示するためのヘルパー関数が含まれています。

In [ ]:
def invoke_claude(
    prompt, 
    model_id=CLAUDE_37_SONNET_MODEL_ID, 
    enable_reasoning=False, 
    reasoning_budget=1024,
    temperature=0.7,
    max_tokens=1000
):
    """
    Invoke Claude with or without extended thinking.
    
    Args:
        prompt (str): The prompt to send to Claude
        model_id (str): The model ID to use
        enable_reasoning (bool): Whether to enable extended thinking
        reasoning_budget (int): Token budget for reasoning (min 1024)
        temperature (float): Temperature for generation (0.0-1.0)
        max_tokens (int): Maximum tokens to generate
        
    Returns:
        dict: The complete API response
    """
    # Create system prompt and messages
    system_prompt = [{"text": "You're a helpful AI assistant."}]
    
    messages = [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ]
    
    # Base request parameters
    request_params = {
        "modelId": model_id,
        "messages": messages,
        "system": system_prompt,
        "inferenceConfig": {
            "temperature": temperature,
            "maxTokens": max_tokens
        }
    }
    
    # Add reasoning configuration if enabled
    if enable_reasoning:
        # When using reasoning, temperature must be 1.0
        request_params["inferenceConfig"]["temperature"] = 1.0
        
        # Ensure maxTokens is greater than reasoning_budget
        if max_tokens <= reasoning_budget:
            # Make it just one token more than the reasoning budget
            adjusted_max_tokens = reasoning_budget + 1
            print(f"Info: Increasing maxTokens from {max_tokens} to {adjusted_max_tokens} to exceed reasoning budget")
            request_params["inferenceConfig"]["maxTokens"] = adjusted_max_tokens
        
        request_params["additionalModelRequestFields"] = {
            "reasoning_config": {
                "type": "enabled",
                "budget_tokens": reasoning_budget
            }
        }
    
    # Invoke the model
    start_time = time.time()
    response = bedrock_runtime.converse(**request_params)
    elapsed_time = time.time() - start_time
    
    # Add elapsed time to response for reference
    response["_elapsed_time"] = elapsed_time
    
    return response


def extract_response_content(response):
    """Extract the response content from Claude's API response"""
    if response.get('output', {}).get('message', {}).get('content'):
        content_blocks = response['output']['message']['content']
        for block in content_blocks:
            if 'text' in block:
                return block['text']
    return "No response content found"

def print_section_header(message):
    """Helper function to print a decorative section header"""
    width = 80
    decoration = colored("╔" + "═"*(width-2) + "╗", "blue")
    empty_line = colored("║" + " "*(width-2) + "║", "blue")
    
    # Center the message
    message_length = len(message)
    padding = (width - 2 - message_length) // 2
    message_line = colored("║", "blue") + " "*padding + colored(message, "cyan", attrs=["bold"]) + " "*(width - 2 - padding - message_length) + colored("║", "blue")
    
    bottom = colored("╚" + "═"*(width-2) + "╝", "blue")
    
    print("\n" + decoration)
    print(empty_line)
    print(message_line)
    print(empty_line)
    print(bottom + "\n")

def display_claude_response(response, show_reasoning=False):
    """Display Claude's response in a nicely formatted way with colors"""
    result = extract_response_content(response)
    
    # Calculate costs (approximate)
    input_tokens = response.get('usage', {}).get('inputTokens', 0)
    output_tokens = response.get('usage', {}).get('outputTokens', 0)
    total_tokens = response.get('usage', {}).get('totalTokens', 0)
    
    input_cost = input_tokens * 0.000003  # $3 per million tokens
    output_cost = output_tokens * 0.000015  # $15 per million tokens
    total_cost = input_cost + output_cost
    
    # Create styled HTML output
    metrics_html = f"""
    <div style="background-color: #f0f8ff; padding: 15px; border-radius: 5px; margin: 10px 0;">
        <h3 style="color: #2c5282; margin-top: 0;">📊 Response Metrics</h3>
        <p style="color: #4a5568;"><strong>⏱️ Time:</strong> {response.get('_elapsed_time', 0):.2f} seconds</p>
        <p style="color: #4a5568;"><strong>🔤 Tokens:</strong> {total_tokens:,} total 
           (<span style="color: #38a169;">{input_tokens:,} input</span>, 
            <span style="color: #805ad5;">{output_tokens:,} output</span>)</p>
        <p style="color: #4a5568;"><strong>💰 Estimated cost:</strong> 
           <span style="color: #e53e3e;">${total_cost:.5f}</span></p>
    </div>
    """
    
    response_html = f"""
    <div style="background-color: #fff5f5; padding: 15px; border-radius: 5px; margin: 10px 0;">
        <h3 style="color: #2c5282; margin-top: 0;">🤖 Claude's Response:</h3>
        <div style="color: #4a5568;">{result}</div>
    </div>
    """
    
    # Display the formatted output
    display(HTML(metrics_html))
    display(HTML(response_html))
    
    # Also print colored console output for non-notebook environments
    print(colored("\n=== Response Metrics ===", "blue", attrs=["bold"]))
    print(colored(f"Time: {response.get('_elapsed_time', 0):.2f} seconds", "cyan"))
    print(colored(f"Tokens: {total_tokens:,} total ({input_tokens:,} input, {output_tokens:,} output)", "green"))
    print(colored(f"Estimated cost: ${total_cost:.5f}", "yellow"))
    print(colored("\n=== Claude's Response ===", "blue", attrs=["bold"]))
    print(colored(result, "white"))
    
    return result


## 4. 標準モードと拡張思考モードの比較

では、Claude 3.7 Sonnet の動作を見てみましょう。拡張思考を有効にした場合と有効にしなかった場合のパフォーマンスを比較します。拡張思考がどのような場合に最もメリットをもたらすかを示すために、いくつかの異なるタイプの問題でテストします。

In [ ]:
# Simple example - Capital city question
simple_prompt = "What is the capital of France?"

# Without extended thinking
print_section_header("Calling Claude 3.7 Sonnet WITHOUT extended thinking...")
standard_response = invoke_claude(
    simple_prompt,
    enable_reasoning=False,
    max_tokens=100
)

display_claude_response(standard_response, show_reasoning=False)

# With extended thinking
print_section_header("Calling Claude 3.7 Sonnet WITH extended thinking...")
reasoning_response = invoke_claude(
    simple_prompt,
    enable_reasoning=True,
    reasoning_budget=1024,  # Minimum budget
    max_tokens=100
)

display_claude_response(reasoning_response)

In [ ]:
# More complex example - Sock drawer problem
complex_prompt = """
A man has 53 socks in his drawer: 21 identical blue, 15 identical black and 17 identical red.
The lights are out, and he is completely in the dark. How many socks must he take out to make 100 percent
certain he has at least one pair of black socks?
"""

# Without extended thinking
print("Calling Claude 3.7 Sonnet WITHOUT extended thinking...")
standard_complex_response = invoke_claude(
    complex_prompt,
    enable_reasoning=False,
    max_tokens=300
)

display_claude_response(standard_complex_response, show_reasoning=False)

# With extended thinking
print("\nCalling Claude 3.7 Sonnet WITH extended thinking...")
reasoning_complex_response = invoke_claude(
    complex_prompt,
    enable_reasoning=True,
    reasoning_budget=2048,  # A bit more budget for this problem
    max_tokens=2049  # Ensuring it's at least 1 token more than the reasoning budget
)

display_claude_response(reasoning_complex_response)

## 5. 異なる推論予算のパフォーマンス分析

では、異なる推論予算が靴下引き出し問題における Claude のパフォーマンスにどのように影響するかを体系的に分析してみましょう。4 つの異なる予算サイズをテストします:

- **1,024 トークン**: 最低限必要な予算
- **2,048 トークン**: 中程度の予算
- **4,096 トークン**: 十分な予算
- **8,192 トークン**: 非常に大きな予算

予算サイズごとに、次の項目を測定します:

1. **応答時間**: 応答を取得するのにかかる時間
2. **トークン使用量**: 使用されたトークンの合計数 (入力 + 出力)
3. **コスト**: トークン使用量に基づく推定コスト
4. **効率**: 1 秒あたりに処理されるトークン数

この分析により、コスト、速度、パフォーマンスのバランスが取れた最適な推論予算を見つけることができます。計算タスクに割り当てる適切な CPU 時間を見つけるようなものと考えてください。CPU 時間が少なすぎると、モデルが問題を効果的に解決するための「思考スペース」が不足する可能性があります。CPU 時間が多すぎると、リソースを無駄にしてしまうことになります。

In [ ]:
# Compare different reasoning budgets for the complex problem
print("Testing different reasoning budgets on the sock drawer problem...")

results = []
budgets = [1024, 2048, 4096, 8192, 16384]  # Different budget sizes to test

for budget in budgets:
    print(f"\nTesting with reasoning budget: {budget} tokens")
    response = invoke_claude(
        complex_prompt,
        enable_reasoning=True,
        reasoning_budget=budget,
        max_tokens=300
    )
    
    # Extract metrics
    metrics = {
        'budget': budget,
        'time': response.get('_elapsed_time', 0),
        'input_tokens': response.get('usage', {}).get('inputTokens', 0),
        'output_tokens': response.get('usage', {}).get('outputTokens', 0),
        'total_tokens': response.get('usage', {}).get('totalTokens', 0),
        'cost': (response.get('usage', {}).get('inputTokens', 0) * 0.000003) + 
                (response.get('usage', {}).get('outputTokens', 0) * 0.000015)
    }
    
    results.append(metrics)
    
    # Display brief summary
    print(f"Time: {metrics['time']:.2f}s, Tokens: {metrics['total_tokens']}, Cost: ${metrics['cost']:.5f}")

# Create a DataFrame and display the results
performance_df = pd.DataFrame(results)
display(performance_df)

# Plot the results
plt.figure(figsize=(14, 8))

plt.subplot(2, 2, 1)
plt.plot(performance_df['budget'], performance_df['time'], marker='o')
plt.title('Time vs. Reasoning Budget')
plt.xlabel('Reasoning Budget (tokens)')
plt.ylabel('Time (seconds)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(performance_df['budget'], performance_df['total_tokens'], marker='o')
plt.title('Total Tokens vs. Reasoning Budget')
plt.xlabel('Reasoning Budget (tokens)')
plt.ylabel('Total Tokens')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(performance_df['budget'], performance_df['cost'], marker='o')
plt.title('Cost vs. Reasoning Budget')
plt.xlabel('Reasoning Budget (tokens)')
plt.ylabel('Cost ($)')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
efficiency = performance_df['total_tokens'] / performance_df['time']
plt.plot(performance_df['budget'], efficiency, marker='o')
plt.title('Efficiency (Tokens/Second) vs. Reasoning Budget')
plt.xlabel('Reasoning Budget (tokens)')
plt.ylabel('Tokens per Second')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 拡張思考を使用する場合

当社の実験とパフォーマンス分析に基づいて、拡張思考を使用する場合についていくつかの結論を導き出すことができます。

### 拡張思考の最適な使用例:

1. **複雑な推論タスク**: 数学の問題、論理パズル、および複数ステップの推論は、拡張思考から大きなメリットを得られます。

2. **徹底的な分析を必要とする問題**: クロードが多くの可能性やエッジケースを考慮する必要がある場合。

3. **精度が重要な場合**: 拡張思考は、推論エラーの可能性を減らすことで、一般的に難しい問題の精度を向上させます。

4. **透明性の要件**: クロードの推論プロセスを確認してアプローチを検証する必要がある場合。

### 標準モードで十分な場合:

1. **単純な事実のクエリ**: 「フランスの首都は?」などの簡単な質問の場合、拡張思考は大きなメリットがなくコストを追加します。

2. **クリエイティブなタスク**: クリエイティブなライティング、要約、その他のコンテンツ生成タスクでは、拡張思考からそれほど恩恵を受けられない可能性があります。

3. **時間に敏感なアプリケーション**: 応答速度が重要な場合は、標準モードの方が応答が速くなります。

4. **コストに敏感なアプリケーション**: 拡張思考によりトークンの使用が増加し、コストも増加します。

### 適切な推論予算の見つけ方:

理想的な推論予算は、タスクの複雑さによって異なります:

- **単純な推論タスク**: 1,024～2,048 トークン
- **中程度の複雑さ**: 2,048～4,096 トークン
- **複雑な問題**: 4,096～8,192 トークン
- **非常に複雑な問題**: 8,192 トークン以上

パフォーマンス分析で観察されたように、多くの場合、「スイート スポット」があります:
- 予算が小さすぎると、Claude が複雑な問題を解決するのに十分なスペースが得られない可能性があります
- あるポイントを超えると、予算が大きくなると収益が減少する一方で、コストとレイテンシが増加します
- 効率 (1 秒あたりのトークン数) は、中程度の予算サイズでピークに達し、その後低下する傾向があります

パフォーマンス チャートは、このトレードオフを視覚的に示し、特定のユース ケースに最適な予算を決定するのに役立ちます。

次のノートブックでは、拡張思考をいつ使用するか、またさまざまなタスク タイプに対して推論予算を最適化する方法を決定するための、より体系的なフレームワークについて説明します。